In [43]:
import h2o
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from category_encoders import CountEncoder
from h2o.sklearn import H2OAutoMLClassifier, H2OAutoMLRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, mean_squared_error
import warnings

warnings.filterwarnings(action='ignore')

In [2]:
data = pd.read_csv('/home/tigran/Downloads/code/Data/StateFarm_missing.csv', sep=';')
data.head()

,Customer Lifetime Value,Coverage,Education,EmploymentStatus,Gender,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Response
0,2763.519279,Basic,Bachelor,Employed,F,56274.0,NaN,32.0,5.0,NaN,1.0,No
1,NaN,NaN,Bachelor,Unemployed,F,0.0,NaN,13.0,42.0,NaN,NaN,No
2,NaN,NaN,NaN,Employed,F,48767.0,108.0,NaN,38.0,0.0,NaN,No
3,7645.861827,Basic,Bachelor,NaN,NaN,0.0,106.0,18.0,NaN,NaN,7.0,No
4,2813.692575,Basic,Bachelor,NaN,M,43836.0,73.0,12.0,NaN,NaN,1.0,No


In [4]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('Response', axis=1), 
                                                   data['Response'],
                                                   random_state=42,
                                                   test_size=0.3)

In [5]:
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

In [6]:
pipe = Pipeline([
    ('imp', SimpleImputer(strategy='constant')),
    ('classifier', H2OAutoMLClassifier(preprocessing=None, max_models=10, seed=2022))
])

In [7]:
pipe.fit(X_train, y_train)

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,51 mins 02 secs
H2O_cluster_timezone:,Asia/Yerevan
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.4
H2O_cluster_version_age:,16 days
H2O_cluster_name:,H2O_from_python_tigran_qgo9o6
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.706 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


Pipeline(steps=[('imp', SimpleImputer(strategy='constant')),
                ('classifier', H2OAutoMLClassifier(max_models=10, seed=2022))])

In [11]:
predictions = pipe.predict(X_test)
acc = accuracy_score(y_test, predictions)
print(f'Accuracy on test set: {acc}')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Accuracy on test set: 0.9356913183279743


In [12]:
automl = pipe.named_steps.classifier.estimator
automl.leaderboard

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
GBM_4_AutoML_2_20230514_134308,0.917158,0.16647,0.656846,0.0872583,0.214125,0.0458494
StackedEnsemble_BestOfFamily_1_AutoML_2_20230514_134308,0.917126,0.165008,0.656805,0.0915104,0.214413,0.0459727
StackedEnsemble_AllModels_1_AutoML_2_20230514_134308,0.91673,0.164898,0.658188,0.0900686,0.214234,0.045896
GBM_3_AutoML_2_20230514_134308,0.915965,0.170348,0.655402,0.107334,0.216452,0.0468516
GBM_2_AutoML_2_20230514_134308,0.910796,0.174317,0.647734,0.111763,0.218276,0.0476443
GBM_1_AutoML_2_20230514_134308,0.907392,0.188291,0.617887,0.1468,0.229778,0.052798
XGBoost_3_AutoML_2_20230514_134308,0.905705,0.190535,0.611827,0.137354,0.227777,0.0518823
XGBoost_2_AutoML_2_20230514_134308,0.904401,0.191867,0.607958,0.151464,0.229061,0.0524687
DRF_1_AutoML_2_20230514_134308,0.903171,0.271776,0.646533,0.100838,0.222996,0.0497272
XRT_1_AutoML_2_20230514_134308,0.902794,0.242251,0.639145,0.114161,0.223052,0.0497522


In [13]:
pipe = Pipeline([
    ('imp', SimpleImputer(strategy='constant')),
    ('classifier', H2OAutoMLClassifier(preprocessing=None, exclude_algos=['DRF'], max_models=10, seed=2023))
])

pipe.fit(X_train, y_train)
predictions = pipe.predict(X_test)
acc = accuracy_score(y_test, predictions)
print(f'Accuracy score without DRF model: {acc:.3f}')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Accuracy score without DRF model: 0.936


In [14]:
automl = pipe.named_steps.classifier.estimator
automl.leaderboard

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
GBM_4_AutoML_3_20230514_134826,0.917158,0.16647,0.656846,0.0872583,0.214125,0.0458494
StackedEnsemble_BestOfFamily_1_AutoML_3_20230514_134826,0.916982,0.165118,0.65606,0.0880307,0.214485,0.0460039
StackedEnsemble_AllModels_1_AutoML_3_20230514_134826,0.916499,0.165069,0.659656,0.0902598,0.214412,0.0459726
GBM_3_AutoML_3_20230514_134826,0.915965,0.170348,0.655402,0.107334,0.216452,0.0468516
GBM_2_AutoML_3_20230514_134826,0.910796,0.174317,0.647734,0.111763,0.218276,0.0476443
GBM_5_AutoML_3_20230514_134826,0.910577,0.17477,0.650377,0.109828,0.218888,0.0479118
XGBoost_2_AutoML_3_20230514_134826,0.908219,0.190835,0.60152,0.155216,0.230787,0.0532629
XGBoost_3_AutoML_3_20230514_134826,0.905705,0.190535,0.611827,0.137354,0.227777,0.0518823
GBM_1_AutoML_3_20230514_134826,0.903039,0.193575,0.605443,0.165037,0.233251,0.054406
XGBoost_1_AutoML_3_20230514_134826,0.879413,0.214832,0.535763,0.188834,0.244373,0.059718


In [15]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('Response', axis=1),
                                                   data['Response'],
                                                   test_size=0.3,
                                                   stratify=data['Response'],
                                                   random_state=42)

In [16]:
cat_columns = X_train.select_dtypes(include='object').columns.tolist()
num_columns = X_train.select_dtypes(exclude='object').columns.tolist()

In [17]:
y_train = y_train.values
y_test = y_test.values

In [18]:
num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median'))
])

cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('count', CountEncoder(return_df=False))
])

In [19]:
transformers = [('num', num_pipe, num_columns),
               ('cat', cat_pipe, cat_columns)]

In [20]:
pipe = Pipeline([
    ('tr', ColumnTransformer(transformers=transformers)),
    ('classifier', H2OAutoMLClassifier(max_models=10, seed=2023, preprocessing=None))
])


In [21]:
pipe.fit(X_train, y_train)
predictions = pipe.predict(X_test)
acc = accuracy_score(y_test, predictions)
print(f'Accuracy score is: {acc:.4f}')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Accuracy score is: 0.9361


In [22]:
automl = pipe.named_steps.classifier.estimator
automl.leaderboard

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_1_AutoML_4_20230514_135621,0.914291,0.175417,0.616306,0.0976391,0.222158,0.0493543
GBM_3_AutoML_4_20230514_135621,0.913709,0.17647,0.619919,0.0981178,0.221492,0.0490588
StackedEnsemble_AllModels_1_AutoML_4_20230514_135621,0.912481,0.171994,0.622973,0.0929557,0.219271,0.0480796
GBM_4_AutoML_4_20230514_135621,0.911544,0.173679,0.626385,0.0921948,0.218812,0.0478787
XGBoost_2_AutoML_4_20230514_135621,0.909036,0.192365,0.580027,0.144613,0.233408,0.0544793
GBM_2_AutoML_4_20230514_135621,0.907948,0.179141,0.62779,0.0964045,0.223048,0.0497503
XGBoost_3_AutoML_4_20230514_135621,0.906311,0.193619,0.579664,0.133444,0.232715,0.0541562
GBM_1_AutoML_4_20230514_135621,0.904567,0.193726,0.59069,0.139944,0.234298,0.0548958
XRT_1_AutoML_4_20230514_135621,0.9008,0.294252,0.618559,0.11167,0.225864,0.0510144
DRF_1_AutoML_4_20230514_135621,0.898023,0.316652,0.609564,0.104129,0.226414,0.0512631


In [23]:
num_pipe = Pipeline([
    ('imputer', SimpleImputer())
])
cat_pipe = Pipeline([
    ('imputer', SimpleImputer())
])

In [24]:
transformers = [('num', num_pipe, num_columns),
               ('cat', cat_pipe, cat_columns)]

In [25]:
pipe = Pipeline([
    ('tr', ColumnTransformer(transformers=transformers)),
    ('classifier', H2OAutoMLClassifier(preprocessing=None, max_models=10, seed=2023))
])


In [27]:
hyperparams_dct = {
    'tr__num__imputer__strategy': ['mean', 'median', 'constant'],
    'tr__cat__imputer__strategy': ['most_frequent', 'constant']
}

In [28]:
grid = GridSearchCV(pipe, cv=3, param_grid=hyperparams_dct)

In [29]:
grid.fit(X_train, y_train)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |█████████████████████████████████

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |█████████████████████████████████

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tr',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer())]),
                                                                         ['Customer '
                                                                          'Lifetime '
                                                                          'Value',
                                                                          'Income',
                                                                          'Monthly '
                                                                          'Premium '
                                                                          'Auto',
                                                                          'Months '
                                                                          'Since '
                                                                          'Last '
                                                                          'Claim',
                                                                          'Months '
                                                                          'Since '
                                                                          'Policy '
                                                                          'Inception',
                                                                          'Number '
                                                                          'of '
                                                                          'Open '
                                                                          'Complaints',
                                                                          'Number '
                                                                          'of '
                                                                          'Policies']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer())]),
                                                                         ['Coverage',
                                                                          'Education',
                                                                          'EmploymentStatus',
                                                                          'Gender'])])),
                                       ('classifier',
                                        H2OAutoMLClassifier(max_models=10,
                                                            seed=2023))]),
             param_grid={'tr__cat__imputer__strategy': ['most_frequent',
                                                        'constant'],
                         'tr__num__imputer__strategy': ['mean', 'median',
                                                        'constant']})

In [30]:
predictions = grid.predict(X_test)
acc = accuracy_score(y_test, predictions)
print(f'Accuracy score is {acc:.4f}')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Accuracy score is 0.9393


In [31]:
best = grid.best_estimator_
predictions = best.predict(X_test)
acc = accuracy_score(y_test, predictions)
print(f'Accuracy score of best estimator {acc:.3f}')
print(f'Best hyperparams are {grid.best_params_}')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Accuracy score of best estimator 0.939
Best hyperparams are {'tr__cat__imputer__strategy': 'constant', 'tr__num__imputer__strategy': 'constant'}


In [32]:
results = pd.DataFrame(grid.cv_results_)
table = results.pivot_table(values=['mean_test_score'],
                           index=['param_tr__cat__imputer__strategy', 'param_tr__num__imputer__strategy'])
table

mean_test_score
param_tr__cat__imputer__strategy param_tr__num__imputer__strategy                 
constant                         constant                                 0.934539
                                 mean                                     0.933678
                                 median                                   0.932300
most_frequent                    constant                                 0.933678
                                 mean                                     0.933506
                                 median                                   0.933161

In [34]:
best = grid.best_estimator_['classifier']
automl = best.estimator
automl.leaderboard

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_1_AutoML_23_20230514_140836,0.914549,0.170428,0.622359,0.0949587,0.218375,0.0476877
XGBoost_2_AutoML_23_20230514_140836,0.914378,0.18709,0.601442,0.127529,0.228967,0.0524257
GBM_4_AutoML_23_20230514_140836,0.914224,0.172731,0.625006,0.099156,0.218144,0.0475867
StackedEnsemble_AllModels_1_AutoML_23_20230514_140836,0.914131,0.170388,0.624079,0.0953391,0.218223,0.0476213
GBM_3_AutoML_23_20230514_140836,0.91144,0.17643,0.622389,0.097158,0.220683,0.0487011
GBM_2_AutoML_23_20230514_140836,0.911258,0.177371,0.630531,0.104509,0.221369,0.0490043
GBM_1_AutoML_23_20230514_140836,0.909979,0.187905,0.603359,0.140128,0.231271,0.0534862
XGBoost_3_AutoML_23_20230514_140836,0.908303,0.189471,0.59513,0.122664,0.230151,0.0529694
XRT_1_AutoML_23_20230514_140836,0.902773,0.198172,0.625186,0.119775,0.225161,0.0506973
DRF_1_AutoML_23_20230514_140836,0.898538,0.294601,0.61671,0.0938272,0.226443,0.0512764


In [35]:
data = pd.read_csv('/home/tigran/Downloads/code/Data/Flats_missing.csv', sep=';', decimal=',')
data.head()

,Rooms_Number,District,Stor,Storeys,Space_Total,Space_Living,Space_Kitchen,Balcon_Num,Lodgee_Num,lat,Long,Cost_KV
0,1,Заельцовский,13,17.0,54.1,18.0,21.2,0.0,1,55.0725,82.9069,50831.79298
1,1,Заельцовский,10,17.0,54.5,18.0,21.1,0.0,1,55.0725,82.9069,52000.00000
2,1,Центральный,8,17.0,37.0,0.0,0.0,0.0,0,55.0725,82.9068,87837.83784
3,1,Центральный,2,17.0,42.0,0.0,0.0,0.0,0,55.0725,82.9068,90238.09524
4,1,Центральный,13,17.0,28.0,0.0,0.0,0.0,0,55.0725,82.9068,110714.28570


In [38]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('Cost_KV', axis=1),
                                                   data['Cost_KV'],
                                                   test_size=0.3,
                                                   random_state=42)

In [39]:
X_train, X_test = X_train.values, X_test.values
y_train, y_test = y_train.values, y_test.values

In [40]:
pipe = Pipeline([
    ('imp', SimpleImputer(strategy='constant')),
    ('regressor', H2OAutoMLRegressor(preprocessing=None, max_models=10, seed=2022))
])

In [42]:
pipe.fit(X_train, y_train)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


Pipeline(steps=[('imp', SimpleImputer(strategy='constant')),
                ('regressor', H2OAutoMLRegressor(max_models=10, seed=2022))])

In [44]:
predictions = pipe.predict(X_test)
rms = mean_squared_error(y_test, predictions, squared=False)
print(f'RMSE: {rms:.3f}')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
RMSE: 9483.515


In [45]:
automl = pipe.named_steps.regressor.estimator
automl.leaderboard

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
StackedEnsemble_AllModels_1_AutoML_25_20230514_142102,11109.4,1.23418e+08,6734.8,0.181944,1.23418e+08
StackedEnsemble_BestOfFamily_1_AutoML_25_20230514_142102,11160.9,1.24565e+08,6771.15,0.182826,1.24565e+08
GBM_4_AutoML_25_20230514_142102,11227.1,1.26049e+08,6851.11,0.184222,1.26049e+08
GBM_1_AutoML_25_20230514_142102,11304.2,1.27785e+08,6924,0.184864,1.27785e+08
GBM_3_AutoML_25_20230514_142102,11348.7,1.28793e+08,6961.28,0.186158,1.28793e+08
DRF_1_AutoML_25_20230514_142102,11358.3,1.2901e+08,6921.17,0.185917,1.2901e+08
XRT_1_AutoML_25_20230514_142102,11360.2,1.29053e+08,6908.06,0.185547,1.29053e+08
GBM_2_AutoML_25_20230514_142102,11395,1.29846e+08,7012.8,0.186889,1.29846e+08
XGBoost_3_AutoML_25_20230514_142102,11622.4,1.35079e+08,7163.02,0.190612,1.35079e+08
XGBoost_2_AutoML_25_20230514_142102,11628.4,1.3522e+08,7110.53,0.190115,1.3522e+08


In [47]:
h2o.get_model('DRF_1_AutoML_25_20230514_142102')

Model Details
=============
H2ORandomForestEstimator : Distributed Random Forest
Model Key: DRF_1_AutoML_25_20230514_142102


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    47                 47                          9.67225e+06            20           20           20            14165         19153         16379.9

ModelMetricsRegression: drf
** Reported on train data. **

MSE: 133451062.30450423
RMSE: 11552.10207297807
MAE: 7012.20308915824
RMSLE: 0.18813638507786923
Mean Residual Deviance: 133451062.30450423

ModelMetricsRegression: drf
** Reported on cross-validation data. **

MSE: 129010212.29656787
RMSE: 11358.266253991755
MAE: 6921.1746908948635
RMSLE: 0.18591658290574456
Mean Residual Deviance: 129010212.29656787

Cross-Validation Metrics Summary: 
                        mean        sd          cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  ----------  ----------  ------------  ------------  ------------  ------------  ------------
mae                     6921.18     105.204     7100.1        6859.38       6833.76       6892.95       6919.68
mean_residual_deviance  1.2901e+08  3.7263e+07  1.83525e+08   9.5562e+07    9.23768e+07   1.36371e+08   1.37216e+08
mse                     1.2901e+08  3.7263e+07  1.83525e+08   9.5562e+07    9.23768e+07   1.36371e+08   1.37216e+08
r2                      0.491837    0.0663251   0.407947      0.559053      0.559854      0.473099      0.459232
residual_deviance       1.2901e+08  3.7263e+07  1.83525e+08   9.5562e+07    9.23768e+07   1.36371e+08   1.37216e+08
rmse                    11265.1     1622.74     13547.1       9775.58       9611.29       11677.8       11713.9
rmsle                   0.185894    0.00323014  0.190868      0.187098      0.182804      0.183567      0.185134

Scoring History: 
    timestamp            duration    number_of_trees    training_rmse    training_mae    training_deviance
--  -------------------  ----------  -----------------  ---------------  --------------  -------------------
    2023-05-14 14:22:20  37.140 sec  0                  nan              nan             nan
    2023-05-14 14:22:21  37.742 sec  5                  13696            8473.33         1.8758e+08
    2023-05-14 14:22:21  38.370 sec  10                 12417.9          7771.04         1.54205e+08
    2023-05-14 14:22:22  38.993 sec  15                 12020.3          7429.65         1.44488e+08
    2023-05-14 14:22:22  39.662 sec  20                 11824.6          7267.98         1.39822e+08
    2023-05-14 14:22:23  40.333 sec  25                 11778.8          7192.46         1.3874e+08
    2023-05-14 14:22:24  41.059 sec  30                 11693.1          7134.75         1.36728e+08
    2023-05-14 14:22:25  41.886 sec  35                 11672            7092.66         1.36235e+08
    2023-05-14 14:22:25  42.620 sec  40                 11631.4          7050.51         1.35289e+08
    2023-05-14 14:22:26  43.333 sec  45                 11572.9          7023.53         1.33932e+08
    2023-05-14 14:22:27  43.714 sec  47                 11552.1          7012.2          1.33451e+08

Variable Importances: 
variable    relative_importance    scaled_importance    percentage
----------  ---------------------  -------------------  ------------
C10         6.10523e+13            1                    0.178531
C5          5.01753e+13            0.82184              0.146724
C4          4.43977e+13            0.727208             0.129829
C11         4.43182e+13            0.725905             0.129597
C2          4.26208e+13            0.698102             0.124633
C6          2.85381e+13            0.467437             0.0834522
C3          2.27958e+13        

In [48]:
gbm = automl.get_best_model(algorithm='gbm')
gbm

Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_4_AutoML_25_20230514_142102


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    76                 76                          247662                 10           10           10            28            570           254.921

ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 91071507.02231532
RMSE: 9543.139264535299
MAE: 6043.618145537464
RMSLE: 0.16343547744501866
Mean Residual Deviance: 91071507.02231532

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 126048644.99227266
RMSE: 11227.138771399981
MAE: 6851.108989652715
RMSLE: 0.1842217450820712
Mean Residual Deviance: 126048644.99227266

Cross-Validation Metrics Summary: 
                        mean        sd           cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  ----------  -----------  ------------  ------------  ------------  ------------  ------------
mae                     6852.16     77.988       6987.18       6842.61       6795.46       6803.16       6832.37
mean_residual_deviance  1.2612e+08  3.54715e+07  1.77957e+08   9.44851e+07   9.10095e+07   1.32799e+08   1.34352e+08
mse                     1.2612e+08  3.54715e+07  1.77957e+08   9.44851e+07   9.10095e+07   1.32799e+08   1.34352e+08
r2                      0.502745    0.0612291    0.42591       0.564023      0.566369      0.486901      0.470521
residual_deviance       1.2612e+08  3.54715e+07  1.77957e+08   9.44851e+07   9.10095e+07   1.32799e+08   1.34352e+08
rmse                    11143       1562.58      13340         9720.34       9539.89       11523.9       11591
rmsle                   0.184233    0.00254436   0.187482      0.186091      0.18168       0.18193       0.18398

Scoring History: 
    timestamp            duration    number_of_trees    training_rmse    training_mae    training_deviance
--  -------------------  ----------  -----------------  ---------------  --------------  -------------------
    2023-05-14 14:22:51  7.706 sec   0                  15806.5          11372.9         2.49845e+08
    2023-05-14 14:22:51  7.814 sec   5                  13387.5          9212.78         1.79224e+08
    2023-05-14 14:22:51  7.916 sec   10                 12029            7950.57         1.44697e+08
    2023-05-14 14:22:51  8.021 sec   15                 11274.8          7258.04         1.2712e+08
    2023-05-14 14:22:52  8.122 sec   20                 10813.8          6880.9          1.16939e+08
    2023-05-14 14:22:52  8.215 sec   25                 10529            6659.97         1.10859e+08
    2023-05-14 14:22:52  8.309 sec   30                 10327.8          6503.08         1.06663e+08
    2023-05-14 14:22:52  8.397 sec   35                 10152.4          6393.7          1.0307e+08
    2023-05-14 14:22:52  8.488 sec   40                 10049.9          6325.45         1.01e+08
    2023-05-14 14:22:52  8.567 sec   45                 9978.45          6292.06         9.95694e+07
    2023-05-14 14:22:52  8.651 sec   50                 9905.2           6242.32         9.8113e+07
    2023-05-14 14:22:52  8.742 sec   55                 9813.92          6185.39         9.6313e+07
    2023-05-14 14:22:52  8.826 sec   60                 9756.76          6151.73         9.51944e+07
    2023-05-14 14:22:52  8.906 sec   65                 9713.66          6130.32         9.43552e+07
    2023-05-14 14:22:52  8.993 sec   70                 9631.61          6086.08         9.2768e+07
    2023-05-14 14:22:52  9.075 sec   75                 9565.61          6052.45         9.15009e+07
    2023-05-14 14:22:52  9.095 sec   76                 9543.14     

In [49]:
gbm.params

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'GBM_4_AutoML_25_20230514_142102',
   'type': 'Key<Model>',
   'URL': '/3/Models/GBM_4_AutoML_25_20230514_142102'},
  'input': None},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'AutoML_25_20230514_142102_training_py_154_sid_9995',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/AutoML_25_20230514_142102_training_py_154_sid_9995'},
  'input': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'AutoML_25_20230514_142102_training_py_154_sid_9995',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/AutoML_25_20230514_142102_training_py_154_sid_9995'}},
 'validation_frame': {'default': None, 'actual': None, 'input': None},
 'nfolds': {'default': 0, 'actual': 5, 'input': 5}